In [ ]:
!pip install tensorflow

The above code sucks !! screw it lets start all over with something more understandable 

In [ ]:
# Importing dependencies
# import tensorflow as tf
import os
import cv2
import imghdr
import tensorflow as tf
# Limiting the GPU use 
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

### Loading the data 

In [ ]:
# Checking the data size 
data_dir=r'/kaggle/input/percy-data/Percy Jackson Cast'
DIR=os.listdir(data_dir)
# data_dir=r'/kaggle/input/celebrity-face-image-dataset/Celebrity Faces Dataset'
# DIR=os.listdir(data_dir)
count=0
character={}
for char in DIR:
    character[char]=len(os.path.join(data_dir,f'{char}'))
print(character)
# print((character))
character_name=character.keys()
print(character_name)

### Copying and cleaning the data

In [ ]:
!cp -r /kaggle/input/percy-data/Percy\ Jackson\ Cast /kaggle/working/

In [ ]:
data_dir=r'/kaggle/working/Percy Jackson Cast'

image_exts=['jpg','jpeg','bmp','png']
for char in DIR:
    for img in os.listdir(os.path.join(data_dir,f'{char}')):
        img_path=os.path.join(os.path.join(data_dir,f'{char}'),img)
        
        try :
            cv2.imread(img_path)
            tip=imghdr.what(img_path)
#             print(img_path,tip)
            if tip not in image_exts:
                print(f"The extension {tip} not available ! ")
                os.remove(img_path)
        except Exception as e:
            print(f"Issue loading the image {img} in {char} directory !")
#             Manual way but  the tips doesnt work that well ! Also in Kaggle you cant change the dataset files . They are read only :)"")!

In [ ]:
# Using the built-in func to process the data and build a data pipeline ! 
import numpy as np 
from matplotlib import pyplot as plt 
data=tf.keras.utils.image_dataset_from_directory(data_dir)
# Gets in the labels ,preprocesses, resizes the images for you !
data.class_names

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split



# Function to resize images
def resize_images(images, new_size):
    resized_images = [cv2.resize(img, new_size) for img in images]
    return np.array(resized_images)

# Assuming data_iterator2 contains all your data
data_iterator2 = data.as_numpy_iterator()

# Extract all batches from data_iterator2
all_batches = [data_iterator2.next() for _ in range(len(data))]

# Concatenate batches into numpy arrays
x_data = np.concatenate([batch[0] for batch in all_batches], axis=0)
y_data = np.concatenate([batch[1] for batch in all_batches], axis=0)

# Crop faces and resize images
# cropped_x_data = crop_faces(x_data)
new_size = (180,180)
resized_x_data = resize_images(x_data, new_size)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(resized_x_data, y_data, test_size=0.2)

# Check the lengths of the sets
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


In [ ]:
# data_iterator=data.as_numpy_iterator()# --> Converts tensors to numpy array 
# batch=data_iterator.next()
# batch[0].shape
# # BATCH_SIZE=32 
# # Visualizing the data a bit :
import numpy
fig,ax =plt.subplots(ncols=4,nrows=2,figsize=(20,20))
r=0
for idx ,img in enumerate(x_test[5:13]):
    ax[r][(3-idx)].imshow(img.astype(int))
    
    ax[r][(3-idx)].title.set_text(y_test[5+idx])
    if idx==3:
        r+=1





## Deep Learning Model 
### Building the model

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras import backend
from tensorflow.keras.layers import Conv2D, MaxPooling2D , Dense , Flatten , Dropout
# using the VGG16 model for training 
def VGG16(IMG_SIZE=256, channels=3, output_dim=1):
    
    # Initialize the Model
  

    # If 'channels first', update the input_shape
#     if backend.image_data_format() == 'channels_first':
#         input_shape = (channels, IMG_SIZE,IMG_SIZE)
    
    if type(output_dim) is not int:
        raise ValueError('[ERROR] Output dimensions need to be an integer')
    if type(channels) is not int:
        raise ValueError('[ERROR] Channels needs to be an integer')

    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, (3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(128, (3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Converts the 4D output of the Convolutional blocks to a 2D feature which can be read by the Dense layer
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))

    # Output Layer
    model.add(Dense(output_dim, activation='softmax'))
    return model

In [ ]:
model=VGG16(IMG_SIZE=180,output_dim=5)

model.compile('adam', loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

In [ ]:
!rmdir /kaggle/working/logs2

In [ ]:
logdir='/kaggle/working/logs2'
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)#
y_train_cat = tf.keras.utils.to_categorical(y_train, 5)


y_test_cat= tf.keras.utils.to_categorical(y_test, 5)
hist=model.fit(x_train,y_train_cat,epochs=10,validation_data=(x_test,y_test_cat),callbacks=[tensorboard_callback])

## Plotting the 

In [ ]:
fig=plt.figure()
plt.plot(hist.history['loss'],color='teal',label="Loss")
plt.plot(hist.history['val_loss'],color='orange',label="Val_Loss")
fig.suptitle('Loss')
plt.legend(loc="upper left")
plt.show()



In [ ]:
fig=plt.figure()
plt.plot(hist.history['accuracy'],color='teal',label="accuracy")
plt.plot(hist.history['val_accuracy'],color='orange',label="Val_accuracy")
fig.suptitle('Accuracy')
plt.legend(loc="upper left")
plt.show()


### Evaluating the model !

In [ ]:
from tensorflow.keras.metrics import Precision,Recall, BinaryAccuracy
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [ ]:


yhat=model.predict(x_test)
y_test1 = tf.keras.utils.to_categorical(y_test, 5)
pre.update_state(y_test1, yhat)
re.update_state(y_test1, yhat)
acc.update_state(y_test1, yhat)

print(f'Precision = {pre.result().numpy()} ,Recall={re.result().numpy()},Accuracy={acc.result().numpy()}')
    

In [ ]:
from PIL import Image
import requests
from io import BytesIO
url="https://www.j-14.com/wp-content/uploads/2021/05/percyjackson05.jpg?resize=1200%2C1200&quality=86&strip=all"
response = requests.get(url)
img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    
    
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# img = Image.open(BytesIO(response.content))
# img=plt.imread('/kaggle/input/percy-data/Percy Jackson Cast/Annabeth/16480333974527.jpg')
# img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
resize_img=tf.image.resize(img,(180,180))
plt.imshow(resize_img.numpy().astype(int))


In [ ]:
# characters=["Annabeth","Chiron","Luke","Percy"]
yhat=model.predict(np.expand_dims(resize_img,0))
# print(yhat)
print(character.keys())

res=[key for key in character.keys()]
res=sorted(res)
# print(res)
print(np.argmax(yhat[0]))
print(f" The person is {res[np.argmax(yhat[0])]} ")

In [ ]:
# char_path=r'/kaggle/input/percy-jackson-movie-cast-image-classification/Percy Jackson Cast'
# char_dict={}
# characters=[]
# count=0
# for i in os.listdir(char_path):
#     characters.append(i[:])
#     count+=1
#     if count>=10:
#         break
# characters


In [ ]:
from tensorflow.keras.models import load_model
model.save(os.path.join('/kaggle/working/models','image_classifier2.h5'))